<a href="https://colab.research.google.com/github/CPTR295/ML1/blob/main/Support_Vector_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.svm import SVC
from sklearn import datasets

iris = datasets.load_iris()
X=iris["data"][:,(2,3)] #Only need 2 features - lenght/width
y=iris["target"]

setosa_or_versicolor = (y==0)|(y==1)
X=X[setosa_or_versicolor]
y=y[setosa_or_versicolor]

svm_clf=SVC(kernel='linear',C=float("inf"))
svm_clf.fit(X,y)

SVC(C=inf, kernel='linear')

In [2]:
svm_clf.predict([[4.4,5.6]])

array([1])

In [3]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.svm import LinearSVC

X,y = make_moons(n_samples=100,noise=0.15,random_state=42)

polynomial_svm=Pipeline([
    ("poly_features",PolynomialFeatures(degree=3)),
    ("scaler",StandardScaler()),
    ("svm_clf",LinearSVC(C=10,loss="hinge",random_state=42))
])


In [4]:
polynomial_svm.fit(X,y)

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge', random_state=42))])

In [8]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1,cache=True,as_frame=False)

X=mnist['data']
y=mnist['target']

X_train=X[:10000] #As svc takes a long time
X_test=X[65000:]
y_train=y[0:10000]
y_test=y[65000:]

In [9]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [10]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
acc_sc = accuracy_score(y_train,y_pred)

In [11]:
acc_sc

0.9831

In [12]:
import numpy as np
std_sc = StandardScaler()
std_sc.fit(X_train)
X_train_s=std_sc.transform(X_train.astype(np.float32))
X_test_s=std_sc.transform(X_test.astype(np.float32))


In [13]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_s,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [14]:
y_pred = lin_clf.predict(X_train_s)
accuracy_score(y_train, y_pred)

0.9768

In [15]:
svm_g = SVC(gamma="scale")
svm_g.fit(X_train_s,y_train)

SVC()

In [17]:
y_pred = svm_g.predict(X_train_s)
accuracy_score(y_train, y_pred)

0.9838

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal,uniform

param_dis = {"gamma":reciprocal(0.001,0.1),"C":uniform(1,10)}
rnd_search = RandomizedSearchCV(svm_clf,param_dis,n_iter=10,verbose=2,cv=3)
rnd_search.fit(X_train_s,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=9.535032738742665, gamma=0.00116796567905981; total time=  15.0s
[CV] END .....C=9.535032738742665, gamma=0.00116796567905981; total time=   9.1s
[CV] END .....C=9.535032738742665, gamma=0.00116796567905981; total time=  13.2s
[CV] END ......C=5.337926337580074, gamma=0.0650017670853996; total time=  12.0s
[CV] END ......C=5.337926337580074, gamma=0.0650017670853996; total time=  12.6s
[CV] END ......C=5.337926337580074, gamma=0.0650017670853996; total time=   9.1s
[CV] END ....C=1.3287899869410338, gamma=0.02239414730394644; total time=   8.9s
[CV] END ....C=1.3287899869410338, gamma=0.02239414730394644; total time=  22.0s
[CV] END ....C=1.3287899869410338, gamma=0.02239414730394644; total time=  13.8s
[CV] END .....C=1.201120934603649, gamma=0.04011703049910879; total time=  12.1s
[CV] END .....C=1.201120934603649, gamma=0.04011703049910879; total time=   8.8s
[CV] END .....C=1.201120934603649, gamma=0.04011

RandomizedSearchCV(cv=3, estimator=SVC(C=inf, kernel='linear'),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78e6953a2a20>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78e695384c20>},
                   verbose=2)

In [24]:
rnd_search.best_estimator_

SVC(C=np.float64(9.535032738742665), gamma=np.float64(0.00116796567905981),
    kernel='linear')

In [26]:
rnd_search.best_score_

np.float64(0.9112996182641658)

In [28]:
rnd_search.best_estimator_.fit(X_train_s,y_train)

SVC(C=np.float64(9.535032738742665), gamma=np.float64(0.00116796567905981),
    kernel='linear')

In [29]:
y_pred = rnd_search.predict(X_train_s)
accuracy_score(y_train, y_pred)

1.0

In [30]:
y_pred = rnd_search.predict(X_test_s)
accuracy_score(y_test, y_pred)

0.939